<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# リアルタイム レコメンデーション API 環境を構築する
## (Azure Storage & Non-Interactive login enabled)

このリファレンス アーキテクチャは、レコメンデーションシステム構築の完全なライフ サイクルを示しています。このシナリオでは、Azure Databricks を使用して、API として展開するレコメンデーション モデルのトレーニング、適切な Azure リソースの作成を説明します。Azure Cosmos DB、Azure Machine Learning と Azure Kubernetes Service を使用します。

このアーキテクチャは、製品、映画・番組、およびニュースのレコメンデーションを含む多くの一般的なレコメンデーション エンジン シナリオに適用することができます。

### アーキテクチャ
![architecture](https://recodatasets.blob.core.windows.net/images/reco-arch.png "Architecture")

**シナリオ**: とあるメディア企業は、自社のユーザーに映画やビデオのレコメンデーションを提供したいと考えています。パーソナライズされたレコメンデーションを提供することによって、クリックスルー率やサイトの利用率、ユーザー満足度の高い増加を通じて、いくつかのビジネス目標を満たそうとしています。

このリファレンスでは、訓練し、特定のユーザーに対してトップ 10 のレコメンドする映画を提供することが可能なリアルタイムのレコメンデーション サービス API をトレーニングし、展開します。

### コンポーネント
このアーキテクチャには、以下の主要なコンポーネントが含まれます:
* [Azure Storage](https://docs.microsoft.com/ja-jp/azure/storage/) は、トレーニングやテストを行うために入力として利用するデータを格納し、API 介してその他のサービスに提供します。Azure Storage は大量のデータを格納することが可能なため、ここでは元のシナリオでは毎回ダウンロードしていた MovieLens のデータを、最新版も含めて全て格納します。。
* [Azure Databricks](https://docs.microsoft.com/en-us/azure/azure-databricks/what-is-azure-databricks) は、入力データの準備と、Spark クラスター上でレコメンデーション モデルのトレーニング開発環境として使用されます。Azure Databricks はまた、データの処理や機械学習のタスクを実行するためのノートブックで共同作業を行うためのインタラクティブなワークスペースを提供します。
* [Azure Kubernetes Service](https://docs.microsoft.com/en-us/azure/aks/intro-kubernetes)(AKS) は、Kubernetes クラスター上に機械学習モデルサービス API の展開と運用を行うために使用されます。AKS は、コンテナー化されたモデルをホストし、スループット要件を満たすためのスケーラビリティ、ID およびアクセスの管理、ログおよび状態監視を提供します。
* [Azure Cosmos DB](https://docs.microsoft.com/en-us/azure/cosmos-db/introduction) は、ユーザーごとにレコメンドされた映画のトップ 10 を格納するために使用される、グローバル分散型データベース サービスです。Azure Cosmos DB は指定されたユーザーのトップ 10 のレコメンド アイテムの読み取りに対して低レイテンシ (99 パーセンタイルにおいて 10 ms) で提供可能なため、このシナリオに最適です。
* [Azure Machine Learning Service](https://docs.microsoft.com/en-us/azure/machine-learning/service/) は、機械学習モデルのトラッキングと管理、スケーラブルな Azure Kubernetes Service環境にこれらのモデルをパッケージ化、展開するために使用するサービスです。


### 目次
0. [ファイルのインポート](0-ファイルのインポート)
1. [サービスの作成](#1-Service-Creation)
2. [トレーニング](#2-Training)
3. [運用化](#3.-Operationalize-the-Recommender-Service)

## セットアップ

このノートブックは Azure Databricks 上でのみ動作します。Azure Databricks のワークスペースにこのノートブックをインポートする手順については、[この](https://docs.azuredatabricks.net/user-guide/notebooks/notebook-manage.html#import-a-notebook)手順に従ってください。

Azure Databricks のセットアップは、リポジトリ内の [セットアップ ガイド]($./SETUP) で、該当するセクションに従って完了する必要があります。

注意事項: このノートは運用化をサポートするための依存関係を追加する**必要**があります。詳細は[セットアップ ガイド]($./SETUP)を参照してください。

## 0 ファイルのインポート

In [ ]:
import numpy as np
import os
import pandas as pd
import pprint
import shutil
import time, timeit
import urllib
import yaml
import json
import uuid
import matplotlib
import matplotlib.pyplot as plt

from azure.common.client_factory import get_client_from_cli_profile
from azure.mgmt.compute import ComputeManagementClient
import azure.mgmt.cosmosdb
import azureml.core
from azureml.core import Workspace
from azureml.core.run import Run
from azureml.core.experiment import Experiment
from azureml.core.model import Model
from azureml.core.image import ContainerImage
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import Webservice, AksWebservice


import pydocumentdb
import pydocumentdb.document_client as document_client

import pyspark
from pyspark.ml.feature import StringIndexer
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, FloatType, IntegerType, LongType

from reco_utils.dataset import movielens
from reco_utils.dataset.cosmos_cli import find_collection, read_collection, read_database, find_database
from reco_utils.dataset.spark_splitters import spark_random_split
from reco_utils.evaluation.spark_evaluation import SparkRatingEvaluation, SparkRankingEvaluation

print("PySpark version:", pyspark.__version__)
print("Azure SDK version:", azureml.core.VERSION)

## 0.1 Azure Storage の定義

In [ ]:
# Azure Storage のアカウント名
storage_account_name = "<Azure storage account name>"
# Azure Storage のアクセス キー（コンテナやBLOBのSASでは無いので注意）
storage_account_access_key = "<Azure storage access key>"

spark.conf.set(
  "fs.azure.account.key."+storage_account_name+".blob.core.windows.net",
  storage_account_access_key)

## 1 サービスの作成
**Subscription ID**の項目を展開したいサブスクリプションIDによって変更してください。

#### このノートブックで作成されるサービス:
1. [Azure ML Service](https://docs.databricks.com/user-guide/libraries.html)
1. [Azure Cosmos DB](https://azure.microsoft.com/ja-jp/services/cosmos-db/)
1. [Azure Container Registery](https://docs.microsoft.com/ja-jp/azure/container-registry/)
1. [Azure Container Instances](https://docs.microsoft.com/ja-jp/azure/container-instances/)
1. [Azure Application Insights (Azure Monitor)](https://azure.microsoft.com/ja-jp/services/monitor/)
1. [Azure Storage](https://docs.microsoft.com/ja-jp/azure/storage/common/storage-account-overview)
1. [Azure Key Vault](https://azure.microsoft.com/ja-jp/services/key-vault/)
1. [Azure Kubernetes Service (AKS)](https://azure.microsoft.com/ja-jp/services/kubernetes-service/)

In [ ]:
# サービス名称の設定
# (訳注) オリジナルは uuid.uuid4() で毎回ランダムな UUID を生成しているため、再実行のたびに新たなリソースグループが作成されるため廃止
#short_uuid = str(uuid.uuid4())[:4]
# (訳注) 変更後は4桁の英数字を任意で選択する方式に変更
short_uuid = "911c"
prefix = "reco" + short_uuid
data = "mvl"
algo = "als"

# Cosmos DB のシークレット ファイルの保管場所
secrets_path = '/dbfs/FileStore/dbsecrets.json'
ws_config_path = '/dbfs/FileStore'

# 利用するサブスクリプション ID に変更
subscription_id = "<Azure Subscription ID>"


In [ ]:
# リソースグループとワークスペース名の設定
resource_group = prefix + "_" + data
workspace_name = prefix + "_"+data+"_aml"
# リージョンの設定
workspace_region = "<Azure ML region name>"
print("Resource group:", resource_group)

# カラムの設定
userCol = "UserId"
itemCol = "MovieId"
ratingCol = "Rating"

# Cosmos DB の構成値
location = workspace_region
account_name = resource_group + "-ds-sql"
# Cosmos DB の account_name は "_" が利用できず、31文字未満に制限される
account_name = account_name.replace("_","-")[0:min(31,len(prefix))]
DOCUMENTDB_DATABASE = "recommendations"
DOCUMENTDB_COLLECTION = "user_recommendations_" + algo

# Azure ML の構成値
history_name = 'spark-ml-notebook'
# 注: アセット名はスペースを含まない小文字英数のみで、30文字未満に制限
model_name = data+"-"+algo+"-reco.mml"
service_name = data + "-" + algo
experiment_name = data + "_"+ algo +"_Experiment"
# AKS の名称は16文字以下の "-" を含む英数のみが利用可能
aks_name = prefix.replace("_","-")[0:min(12,len(prefix))] + '-aks'
# コンテナの名前を追加
container_image_name = '-'.join([data, algo])

train_data_path = data + "Train"
test_data_path = data + "Test"

### 1.1 Azure ML ワークスペースのインポートまたは作成:
このコマンドは指定された Azure ML ワークスペースが存在するか否かを確認し、存在しない場合には作成を行います。
またこのノートブックでは Azure ML ワークスペースへのアタッチをサービス プリンシパルで行います。サービス プリンシパルの AAD への登録方法は[こちら](https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/manage-azureml-service/authentication-in-azureml/authentication-in-azure-ml.ipynb)をご参照ください。

In [ ]:
from azureml.core.authentication import ServicePrincipalAuthentication

# サービス プリンシパルのパスワードは本番では平文で格納しないように注意（Databricks の変数を利用する）
svc_pr = ServicePrincipalAuthentication(
    tenant_id = "<Service Principal Tenant ID>",
    service_principal_id = "<Service Principal ID>",
    service_principal_password = "<Service Principal Password>"
)

try:
  # 既存の AML ワークスペースを利用
  ws = Workspace(
    subscription_id = subscription_id,
    resource_group = resource_group,
    workspace_name = workspace_name,
    auth=svc_pr
    )
  print("Found workspace {} at location {}".format(ws.name, ws.location))
except :
  # 既存の AML ワークスペースが見つからない場合には作成
  ws = Workspace.create(name = workspace_name,
                      subscription_id = subscription_id,
                      resource_group = resource_group, 
                      location = workspace_region,
                      exist_ok=True)
  # サブスクリプションID、リソースグループ名、ワークスペース名を永続化させるために aml_config/config.json に記録
  ws.write_config(ws_config_path)

### 1.2 レコメンデーション結果の保存先として Cosmos DB リソースを作成:
このノートブックでは、Azure Client をサービス プリンシパルで作成しています。詳しい情報は[こちら](https://docs.microsoft.com/ja-jp/azure/python/python-sdk-azure-authenticate?view=azure-python)をご参照ください。

In [ ]:
from azure.common.client_factory import get_client_from_json_dict
from azure.mgmt.compute import ComputeManagementClient

# az ad sp create-for-rbac --name "MY-PRINCIPAL-NAME" --password "STRONG-SECRET-PASSWORD" で生成された内容を転記
# サービス プリンシパルのパスワードは本番では平文で格納しないように注意（Databricks の変数を利用する）
config_dict = {
  "clientId": "<Generated by az cli>",
  "clientSecret": "<Generated by az cli>",
  "subscriptionId": "<Generated by az cli>",
  "tenantId": "<Generated by az cli>",
  "activeDirectoryEndpointUrl": "<Generated by az cli>",
  "resourceManagerEndpointUrl": "<Generated by az cli>",
  "activeDirectoryGraphResourceId": "<Generated by az cli>",
  "sqlManagementEndpointUrl": "<Generated by az cli>",
  "galleryEndpointUrl": "<Generated by az cli>",
  "managementEndpointUrl": "<Generated by az cli>"
}

In [ ]:
## (訳注) ここで認証エラーが表示される場合には、scripts/prepare_databricks_for_o16n.sh に記載されているライブラリのバージョン指定でライブラリのインストールを行ってください
## 複数のサブスクリプションを持っているユーザーの場合に、subscription_id を明示的に渡す
client = get_client_from_json_dict(azure.mgmt.cosmosdb.CosmosDB, config_dict)

async_cosmosdb_create = client.database_accounts.create_or_update(
    resource_group,
    account_name,
    {
        'location': location,
        'locations': [{
            'location_name': location
        }]
    }
)
account = async_cosmosdb_create.result()

my_keys = client.database_accounts.list_keys(
    resource_group,
    account_name
)

master_key = my_keys.primary_master_key
endpoint = "https://" + account_name + ".documents.azure.com:443/"

# Cosmos DB クライアント
client = document_client.DocumentClient(endpoint, {'masterKey': master_key})

if find_database(client, DOCUMENTDB_DATABASE) == False:
    db = client.CreateDatabase({ 'id': DOCUMENTDB_DATABASE })
else:
    db = read_database(client, DOCUMENTDB_DATABASE)
# コレクションのオプションを作成(RUsが11000となっているので注意！)
options = {
    'offerThroughput': 11000
}

# コレクションの作成
collection_definition = { 'id': DOCUMENTDB_COLLECTION, 'partitionKey': {'paths': ['/id'],'kind': 'Hash'} }
if find_collection(client,DOCUMENTDB_DATABASE,  DOCUMENTDB_COLLECTION) ==False:
    collection = client.CreateCollection(db['_self'], collection_definition, options)
else:
    collection = read_collection(client, DOCUMENTDB_DATABASE, DOCUMENTDB_COLLECTION)

In [ ]:
secrets = {
  "Endpoint": endpoint,
  "Masterkey": master_key,
  "Database": DOCUMENTDB_DATABASE,
  "Collection": DOCUMENTDB_COLLECTION,
  "Upsert": "true"
}
with open(secrets_path, "w") as file:
    json.dump(secrets, file)

## 2 トレーニング

次に、ここでは[MovieLens](https://grouplens.org/datasets/movielens/)のデータセットを使用して、[Alternating Least Squares model (交互最小二乗法モデル)](https://spark.apache.org/docs/2.2.0/ml-collaborative-filtering.html)におけるトレーニングを行います。

In [ ]:
# トップ k 個のアイテムをレコメンド
TOP_K = 100

# Movielens のデータサイズを指定: 100k, 1m, 10m, 20m or 27m
MOVIELENS_DATA_SIZE = '1m'

### 2.1 MovieLens データセットのダウンロード

In [ ]:
# MOVIELENS_DATA_SIZE の指定によってダウンロードするファイルを変更。未指定時は 100k を選択
if MOVIELENS_DATA_SIZE == '100k':
  input_file_name = "ml-100k.csv"
elif MOVIELENS_DATA_SIZE == '1m':
  input_file_name = "ml-1m.csv"
elif MOVIELENS_DATA_SIZE == '10m':
  input_file_name = "ml-10m.csv"
elif MOVIELENS_DATA_SIZE == '20m':
  input_file_name = "ml-20m.csv"
elif MOVIELENS_DATA_SIZE == '27m':
  input_file_name = "ml-27m.csv"
else:
  input_file_name = "ml-100k.csv"

# コンテナ名とファイル形式を定義（カンマ区切り、タブ区切りはどちらも csv を指定）
input_container_location = "wasbs://<container name>@<Azure Storage Account Name>.blob.core.windows.net/"
input_file_location =  input_container_location + input_file_name
file_type = "csv"

# 注: データフレームベース API for ALS は現在、整数のユーザー、アイテムIDのみをサポートします。
schema = StructType(
    (
        StructField("UserId", IntegerType()),
        StructField("MovieId", IntegerType()),
        StructField("Rating", FloatType()),
        StructField("Timestamp", LongType()),
    )
)

# CSV の区切り文字を変更する場合には、.option("delimiter", "\t") のように指定する
data = spark.read.format(file_type).schema(schema).option("header", "true").load(input_file_location)
data.show()

### 2.2 トレーニング、テスト用のデータに分割
データを分割するにはランダム、年代順、層別など、いくつかの方法があり、実世界ではユースケースによって適合するものが異なります。この例ではランダムに分割しますが、分割についての詳細は[こちらのガイド](https://github.com/Microsoft/Recommenders/blob/master/notebooks/01_data/data_split.ipynb)を参照してください。

In [ ]:
# （訳注）トレーニング、テスト用に 75:25 の比率で分割。ランダムシードが 123 で設定されているため、毎回同じ分割方法になります
train, test = spark_random_split(data, ratio=0.75, seed=123)
print ("N train", train.cache().count())
print ("N test", test.cache().count())

### 2.3 トレーニングデータで ALS モデルをトレーニングし、テストデータでトップ k 個のレコメンドを取得

映画の評価を予測するには、ユーザーの明示的なフィードバックとセットとなった評価データを使用します。モデルの推論を行うためのハイパーパラメータについては、[このページ](http://mymedialite.net/examples/datasets.html) の内容に基づいています。

ほとんどの状況下では、ハイパーパラメーターの探索と、いくつかの条件に基づいて最適な設定を選択したいと考えるでしょう。このプロセスの詳細については、[ここ](../04_model_select_and_optimize/hypertune_spark_deep_dive.ipynb) にある Deep Dive の追加情報を参照してください。

In [ ]:
header = {
    "userCol": "UserId",
    "itemCol": "MovieId",
    "ratingCol": "Rating",
}


als = ALS(
    rank=10,
    maxIter=15,
    implicitPrefs=False,
    alpha=0.1,
    regParam=0.05,
    coldStartStrategy='drop',
    nonnegative=True,
    **header
)

In [ ]:
# ALSのトレーニング実行
model = als.fit(train)

映画のレコメンドのユース ケースでは、ユーザー自身が評価を行った映画のレコメンドは意味がありません。したがって、評価済みの映画はレコメンドのアイテムから削除されます。

これを実行するためには、すべてのユーザーにすべての映画をお勧めし、トレーニング データセットに存在するユーザーと映画の組み合わせを削除します。

In [ ]:
# 全てのユーザーとアイテムのペアとスコアのクロスジョインを取得
users = train.select('UserId').distinct()
items = train.select('MovieId').distinct()
user_item = users.crossJoin(items)
dfs_pred = model.transform(user_item)

In [ ]:
dfs_pred.show()

In [ ]:
# 表示済みアイテムの除去
dfs_pred_exclude_train = dfs_pred.alias("pred").join(
    train.alias("train"),
    (dfs_pred['UserId'] == train['UserId']) & (dfs_pred['MovieId'] == train['MovieId']),
    how='outer'
)

top_all = dfs_pred_exclude_train.filter(dfs_pred_exclude_train["train.Rating"].isNull()) \
    .select('pred.' + 'UserId', 'pred.' + 'MovieId', 'pred.' + "prediction")

top_all.show()

### 2.4 ALSのパフォーマンス評価

モデルのパフォーマンスを、Precision@K, Recall@K, [MAP](https://en.wikipedia.org/wiki/Evaluation_measures_\(information_retrieval\) または [nDCG](https://en.wikipedia.org/wiki/Discounted_cumulative_gain) のような指標で評価します。レコメンデーションについてどの評価指標が最適かを検討するには、[このガイド](https://github.com/Microsoft/Recommenders/blob/master/notebooks/03_evaluate/evaluation.ipynb)を参照してください。

In [ ]:
test.show()

In [ ]:
rank_eval = SparkRankingEvaluation(test, top_all, k = TOP_K, col_user="UserId", col_item="MovieId", 
                                    col_rating="Rating", col_prediction="prediction", 
                                    relevancy_method="top_k")

In [ ]:
# ランキング指標の評価

print("Model:\tALS",
      "Top K:\t%d" % rank_eval.k,
      "MAP:\t%f" % rank_eval.map_at_k(),
      "NDCG:\t%f" % rank_eval.ndcg_at_k(),
      "Precision@K:\t%f" % rank_eval.precision_at_k(),
      "Recall@K:\t%f" % rank_eval.recall_at_k(), sep='\n')

In [ ]:
# 評価指標の評価

prediction = model.transform(test)
rating_eval = SparkRatingEvaluation(test, prediction, col_user="UserId", col_item="MovieId", 
                                    col_rating="Rating", col_prediction="prediction")

print("Model:\tALS rating prediction",
      "RMSE:\t%.2f" % rating_eval.rmse(),
      "MAE:\t%f" % rating_eval.mae(),
      "Explained variance:\t%f" % rating_eval.exp_var(),
      "R squared:\t%f" % rating_eval.rsquared(), sep='\n')

### 2.5 モデルの保存

In [ ]:
model.write().overwrite().save(model_name)
model_local = "file:" + os.getcwd() + "/" + model_name
dbutils.fs.cp(model_name, model_local, True)

## 3. レコメンデーション サービスを運用化する
望ましいパフォーマンスを持つモデルの構築後は、リアルタイム サービスによって利用される REST エンドポイントとして機能します。ここでは [Azure Cosmos DB](https://azure.microsoft.com/en-us/services/cosmos-db/), [Azure Machine Learning Service](https://azure.microsoft.com/en-us/services/machine-learning-service/) と [Azure Kubernetes Service](https://docs.microsoft.com/en-us/azure/aks/intro-kubernetes) を使用してレコメンデーション サービスを運用します。

### 3.1 Cosmos DB 内にレコメンデーションのルックアップを作成する

最初に、モデルによって予測された各ユーザーのトップ 10 のレコメンドは、Cosmos DB でルックアップ テーブルとして格納されます。実行時に、サービスは Cosmos DB に格納されている事前計算済みのトップ 10 のレコメンドを返します:

In [ ]:
with open(secrets_path) as json_data:
    writeConfig = json.load(json_data)
    recs = model.recommendForAllUsers(10)
    recs.withColumn("id",recs[userCol].cast("string")).select("id", "recommendations."+ itemCol)\
    .write.format("com.microsoft.azure.cosmosdb.spark").mode('overwrite').options(**writeConfig).save()

### 3.2 Azure Machine Learning の構成

次に、Azure Machine Learning Service を使用してモデルのスコアリングイメージを作成し、Azure Kubernetes Service にスケーラブルなコンテナ サービスとして展開します。これを実現するには、**スコアリング スクリプト** と **環境設定** を作成する必要があります。2 つのファイルの内容を次に示します。

スコアリング スクリプトでは、ユーザー IDの入力を与えられたトップ 10 のレコメンドされた映画をルックアップするために Cosmos DB を呼び出します:

In [ ]:
score_sparkml = """

import json
def init(local=False):
    global client, collection
    try:
      # Query them in SQL
      import pydocumentdb.document_client as document_client

      MASTER_KEY = '{key}'
      HOST = '{endpoint}'
      DATABASE_ID = "{database}"
      COLLECTION_ID = "{collection}"
      database_link = 'dbs/' + DATABASE_ID
      collection_link = database_link + '/colls/' + COLLECTION_ID
      
      client = document_client.DocumentClient(HOST, {'masterKey': MASTER_KEY})
      collection = client.ReadCollection(collection_link=collection_link)
    except Exception as e:
      collection = e
def run(input_json):      

    try:
      import json

      id = json.loads(json.loads(input_json)[0])['id']
      query = {'query': 'SELECT * FROM c WHERE c.id = "' + str(id) +'"' } #+ str(id)

      options = {'partitionKey':str(id)}
      document_link = 'dbs/{DOCUMENTDB_DATABASE}/colls/{DOCUMENTDB_COLLECTION}/docs/{0}'.format(id)
      result = client.ReadDocument(document_link, options);
  
    except Exception as e:
        result = str(e)
    return json.dumps(str(result)) #json.dumps({{"result":result}})
"""


with open(secrets_path) as json_data:
    writeConfig = json.load(json_data)
    score_sparkml = score_sparkml.replace("{key}",writeConfig['Masterkey']).replace("{endpoint}",writeConfig['Endpoint']).replace("{database}",writeConfig['Database']).replace("{collection}",writeConfig['Collection']).replace("{DOCUMENTDB_DATABASE}",DOCUMENTDB_DATABASE).replace("{DOCUMENTDB_COLLECTION}", DOCUMENTDB_COLLECTION)

    exec(score_sparkml)

    with open("score_sparkml.py", "w") as file:
        file.write(score_sparkml)

次に、必要な依存関係が記載された環境設定ファイルを作成します:

In [ ]:
%%writefile myenv_sparkml.yml

name: myenv
channels:
  - defaults
dependencies:
  - pip:
    - numpy==1.14.2
    - scikit-learn==0.19.1
    - pandas
    - azureml-core
    - pydocumentdb

作成したモデルを登録します:

In [ ]:
mymodel = Model.register(model_path = model_name, # これはローカルのファイルを指定します
                       model_name = model_name, # これはモデルとして登録されている名前で、パスと名前は同じものを使用します。
                       description = "ADB trained model",
                       workspace = ws)

print(mymodel.name, mymodel.description, mymodel.version)

### 3.3 モデルをサービスとして AKS に展開

#### 3.3.1 モデルサービス用にコンテナを作成:

In [ ]:
# Web サービス用のイメージを作成
models = [mymodel]
runtime = "spark-py"
conda_file = 'myenv_sparkml.yml'
driver_file = "score_sparkml.py"

# イメージの作成
from azureml.core.image import ContainerImage
myimage_config = ContainerImage.image_configuration(execution_script = driver_file, 
                                    runtime = runtime, 
                                    conda_file = conda_file)

image = ContainerImage.create(name = container_image_name,
                                # これはモデル オブジェクトです
                                models = [mymodel],
                                image_config = myimage_config,
                                workspace = ws)

# 作成プロセスの完了を待機
image.wait_for_creation(show_output = True)

#### 3.3.2 コンテナ実行用の AKS クラスタの作成 (このプロセスは 10 から 25分程度かかります):

In [ ]:
from azureml.core.compute import AksCompute, ComputeTarget

# デフォルトの構成を使用 (パラメータを指定してカスタマイズ可能)
prov_config = AksCompute.provisioning_configuration()

# クラスタの作成
aks_target = ComputeTarget.create(workspace = ws, 
                                  name = aks_name, 
                                  provisioning_configuration = prov_config)

aks_target.wait_for_completion(show_output = True)

print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

#### 3.3.3 コンテナ イメージを AKS に展開:

In [ ]:
#Web サービスの構成を設定 (ここではデフォルト設定と App Insight の使用を指定)
aks_config = AksWebservice.deploy_configuration(enable_app_insights=True)

# シングル コマンドを使用して Webservice を作成しますが、直接イメージを使用する方法もあります。
try:
    aks_service = Webservice.deploy_from_image(
      workspace=ws, 
      name=service_name,
      deployment_config = aks_config,
      image = image,
      deployment_target = aks_target
      )
    aks_service.wait_for_deployment(show_output=True)
except Exception:
    aks_service = Webservice.list(ws)[0]

### 3.4 AKS モデル サービスの呼び出し
展開後、ユーザー ID を指定してサービスを呼び出すことができます - サービスは、Cosmos DB でそのユーザーのトップ 10 のレコメンドをルックアップし、結果を送り返します。
次のスクリプトでは、レコメンデーション サービス API を呼び出し、特定のユーザー ID の結果を表示する方法を示しています:

In [ ]:
scoring_url = aks_service.scoring_uri
service_key = aks_service.get_keys()[0]

input_data = '["{\\"id\\":\\"123\\"}"]'.encode()

req = urllib.request.Request(scoring_url,data=input_data)
req.add_header("Authorization","Bearer {}".format(service_key))
req.add_header("Content-Type","application/json")

tic = time.time()
with urllib.request.urlopen(req) as result:
    res = result.readlines()
    print(res)
    
toc = time.time()
t2 = toc - tic
print("Scoring URL : ", scoring_url)
print("Service Key : Bearer ", service_key)
print("Input data : ", input_data.decode())
print(type(input_data))
print("Full run took %.2f seconds" % (toc - tic))